In [1]:
using XLSX
using DataFrames
using OptimalTransportDataIntegration
using DataFrames, CategoricalArrays
using Distributions
using DocStringExtensions
using Parameters
using Printf
using DataFrames
using LinearAlgebra
using Flux
using Distances
using PythonOT
import Statistics: median
using Pkg
using FreqTables
using DataFrames, Statistics, StatsBase, MultivariateStats
using JuMP, Clp
tbl = XLSX.readtable("data.xlsx", 1)   # 1 = première feuille, ou utiliser le nom "Sheet1"
data = DataFrame(tbl)
eltype.(eachcol(data))
for c in names(data)
    data[!, c] = [ 
        x isa Number ? float(x) :                 # si déjà numérique → convertir en Float64
        x isa Missing ? missing :                 # missing → garder
        tryparse(Float64, String(x))              # si string → tenter de parser
        for x in data[!, c]
    ]
end


data = DataFrame(data)

Row,X1,X2,X3,X4,X5,Y,Z,database
,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64
1,36.3258,0.0,0.0,24.16,5.0,3.0,missing,1.0
2,40.2546,0.0,0.0,24.8,4.0,4.0,missing,1.0
3,38.3956,0.0,0.0,21.99,5.0,2.0,missing,1.0
4,35.2909,0.0,0.0,18.65,5.0,3.0,missing,1.0
5,38.0041,0.0,0.0,26.62,5.0,2.0,missing,1.0
6,22.037,0.0,0.0,18.37,5.0,4.0,missing,1.0
7,41.8426,0.0,0.0,19.07,5.0,2.0,missing,1.0
8,35.5592,0.0,0.0,29.74,4.0,2.0,missing,1.0
9,30.3765,0.0,0.0,20.57,5.0,2.0,missing,1.0


Préparation des données

In [2]:
describe(data, :nmissing)



Row,variable,nmissing
,Symbol,Int64
1,X1,0
2,X2,0
3,X3,0
4,X4,0
5,X5,0
6,Y,2848
7,Z,444
8,database,0


In [3]:
for c in [:Y, :Z]
    data[!, c] = coalesce.(data[!, c], -1)
end

for c in [:Y, :Z, :database]
    data[!, c] = convert.(Union{Int64}, data[!, c])
end

In [4]:
println(data)

3292×8 DataFrame
  Row │ X1       X2       X3       X4       X5       Y      Z      database 
      │ Float64  Float64  Float64  Float64  Float64  Int64  Int64  Int64    
──────┼─────────────────────────────────────────────────────────────────────
    1 │ 36.3258      0.0      0.0  24.16        5.0      3     -1         1
    2 │ 40.2546      0.0      0.0  24.8         4.0      4     -1         1
    3 │ 38.3956      0.0      0.0  21.99        5.0      2     -1         1
    4 │ 35.2909      0.0      0.0  18.65        5.0      3     -1         1
    5 │ 38.0041      0.0      0.0  26.62        5.0      2     -1         1
    6 │ 22.037       0.0      0.0  18.37        5.0      4     -1         1
    7 │ 41.8426      0.0      0.0  19.07        5.0      2     -1         1
    8 │ 35.5592      0.0      0.0  29.74        4.0      2     -1         1
    9 │ 30.3765      0.0      0.0  20.57        5.0      2     -1         1
   10 │ 23.989       0.0      0.0  24.52        5.0      4     -1   

In [5]:
dba = subset(data, :database => ByRow(==(1)))
dbb = subset(data, :database => ByRow(==(2)))

cols = names(dba, r"^X")   


#Ylevels = 1:5
#Zlevels = 1:6
Ylevels = 1:4
Zlevels = 1:6
YA = Flux.onehotbatch(dba.Y, Ylevels)
ZB = Flux.onehotbatch(dbb.Z, Zlevels)


6×2848 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  1  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅     1  1  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  1  1  1  1  ⋅     ⋅  ⋅  1  ⋅  1  1  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  1
 ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅

Preparation des covariables en plusieurs format. X: discretisé. Xdm discrétisé en gardant la médiane comme valeur X_hot. One hot encoding

In [6]:


digitize(x, bins) = searchsortedlast.(Ref(bins), x)

XA = dba[:, cols]
XB = dbb[:, cols]

X = Vector{Int}[]
Xmdn = Vector{Float64}[]
cols = ["X1", "X4"]
for col in cols

        b = quantile(data[!, col], collect(0.25:0.25:0.75))
        bins = vcat(-Inf, b, +Inf)

        X1 = digitize(XA[!, col], bins)
        X2 = digitize(XB[!, col], bins)
        push!(X, vcat(X1, X2))

        X1mdn = zeros(Float32, size(X1, 1))
        for i in unique(X1)
            mdn = median(XA[X1 .== i, col])
            X1mdn[X1 .== i] .= mdn
        end

        X2mdn = zeros(Float32, size(X2, 1))
        for i in unique(X2)
            mdn = median(XB[X2 .== i, col])
            X2mdn[X2 .== i] .= mdn
        end

        push!(Xmdn, vcat(X1mdn, X2mdn))

end

cols = ["X2", "X3","X5"]
for col in cols

        X1 = XA[!, col]
        X2 = XB[!, col]
        push!(X, vcat(X1, X2))

        X1mdn = XA[!, col]

        X2mdn = XB[!, col]

        push!(Xmdn, vcat(X1mdn, X2mdn))

end
X = stack(X)
Xmdn = stack(Xmdn)

3292×5 Matrix{Float64}:
 37.8289  27.18    0.0  0.0  5.0
 37.8289  27.18    0.0  0.0  4.0
 37.8289  22.85    0.0  0.0  5.0
 37.8289  19.05    0.0  0.0  5.0
 37.8289  27.18    0.0  0.0  5.0
 25.4155  19.05    0.0  0.0  5.0
 37.8289  19.05    0.0  0.0  5.0
 37.8289  27.18    0.0  0.0  4.0
 29.5688  21.065   0.0  0.0  5.0
 25.4155  27.18    0.0  0.0  5.0
 25.4155  19.05    0.0  0.0  4.0
 32.642   27.18    0.0  0.0  4.0
 37.8289  27.18    0.0  0.0  5.0
  ⋮                          
 29.3854  26.5643  0.0  0.0  6.0
 26.1451  22.6757  1.0  1.0  6.0
 26.1451  22.6757  1.0  0.0  5.0
 26.1451  22.6757  1.0  0.0  6.0
 36.0698  26.5643  1.0  0.0  4.0
 26.1451  22.6757  1.0  0.0  6.0
 36.0698  26.5643  0.0  0.0  6.0
 32.2108  20.7316  1.0  0.0  3.0
 29.3854  19.1001  0.0  0.0  5.0
 36.0698  22.6757  0.0  0.0  5.0
 29.3854  26.5643  0.0  0.0  4.0
 32.2108  19.1001  0.0  0.0  4.0

In [7]:

X_hot = one_hot_encoder(X)


3292×17 Matrix{Int64}:
 0  0  0  1  0  0  0  1  0  0  0  0  0  0  1  0  0
 0  0  0  1  0  0  0  1  0  0  0  0  0  1  0  0  0
 0  0  0  1  0  0  1  0  0  0  0  0  0  0  1  0  0
 0  0  0  1  1  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  1  0  0  0  1  0  0  0  0  0  0  1  0  0
 1  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  1  1  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  1  0  0  0  1  0  0  0  0  0  1  0  0  0
 0  1  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0
 1  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0
 1  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  1  0  0  0  0  1  0  0  0  0  0  1  0  0  0
 0  0  0  1  0  0  0  1  0  0  0  0  0  0  1  0  0
 ⋮              ⋮              ⋮              ⋮  
 0  1  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0
 1  0  0  0  0  0  1  0  1  1  0  0  0  0  0  1  0
 1  0  0  0  0  0  1  0  1  0  0  0  0  0  1  0  0
 1  0  0  0  0  0  1  0  1  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  1  1  0  0  0  0  1  0  0  0
 1  0  0 

OT within

In [8]:

Y = Vector(data.Y)
Z = Vector(data.Z)
XA = transpose(Matrix(dba[:, cols]))
XB = transpose(Matrix(dbb[:, cols]))
base = data.database

indA = findall(base .== 1)
indB = findall(base .== 2)

Xobserv = vcat(X[indA, :], X[indB, :])
Yobserv = vcat(Y[indA], Y[indB])
Zobserv = vcat(Z[indA], Z[indB])

nA = length(indA)
nB = length(indB)

# list the distinct modalities in A and B
indY = [findall(Y[indA] .== m) for m in Ylevels]
indZ = [findall(Z[indB] .== m) for m in Zlevels]

# compute the distance between pairs of individuals in different bases
# devectorize all the computations to go about twice faster only compute norm 1 here


6-element Vector{Vector{Int64}}:
 [576, 838, 1287, 1511, 1949, 2525, 2649, 2757]
 [1, 3, 8, 23, 36, 47, 48, 49, 52, 53  …  2798, 2803, 2804, 2809, 2819, 2820, 2837, 2838, 2840, 2843]
 [5, 7, 9, 10, 11, 12, 19, 22, 24, 26  …  2797, 2801, 2810, 2816, 2825, 2830, 2839, 2841, 2842, 2845]
 [608, 983, 984, 987, 988, 989, 990, 991, 992, 993  …  1590, 1591, 1595, 1596, 1597, 1598, 1599, 1600, 1601, 1603]
 [2, 6, 15, 16, 17, 18, 25, 27, 28, 32  …  2827, 2829, 2831, 2832, 2833, 2834, 2836, 2844, 2846, 2848]
 [4, 13, 14, 20, 21, 29, 31, 34, 35, 38  …  2807, 2808, 2814, 2815, 2818, 2821, 2826, 2828, 2835, 2847]

Distance de Gower

In [9]:
struct GowerDF2 <: PreMetric
    continuous::Vector{Symbol}
    categorical::Vector{Symbol}
    ranges::Dict{Symbol,Float64}   # range par variable continue
end

function GowerDF2(continuous, categorical, df::DataFrame)
    ranges = Dict{Symbol,Float64}()
    for col in continuous
        coldata = skipmissing(df[!, col])
        ranges[col] = maximum(coldata) - minimum(coldata)
     end
    return GowerDF2(continuous, categorical, ranges)
end

function Distances.evaluate(d::GowerDF2, a::DataFrameRow, b::DataFrameRow)
    s = 0.0
    p = length(d.continuous) + length(d.categorical)

    # continues (version correcte Gower)
    for col in d.continuous
        range = d.ranges[col]

        if range == 0 || ismissing(a[col]) || ismissing(b[col])
            contrib = 0.0
        else
            contrib = abs(a[col] - b[col]) / range
        end

        s += contrib
    end

    # catégories
    for col in d.categorical
        if ismissing(a[col]) || ismissing(b[col])
            s += 0.0
        else
            s += (a[col] == b[col] ? 0.0 : 1.0)
        end
    end

    return s / p
end

function pairwise_gower(d::GowerDF2, A::DataFrame, B::DataFrame)
    nA = nrow(A)
    nB = nrow(B)
    D = Matrix{Float64}(undef, nA, nB)

    for i in 1:nA
        row_i = A[i, :]
        for j in 1:nB
            row_j = B[j, :]
            D[i,j] = Distances.evaluate(d, row_i, row_j)
        end
    end

    return D
end

pairwise_gower (generic function with 1 method)

In [10]:

cols = names(dba, r"^X")
for name in ["X2","X3","X5"]
    lev = union(unique(dba[!, name]), unique(dbb[!, name]))
    dba[!, name] = categorical(dba[!, name], levels=lev)
    dbb[!, name] = categorical(dbb[!, name], levels=lev)
end             

# convertir les continues en Float64
for name in ["X1","X4"]
    dba[!, name] = Float64.(dba[!, name])
    dbb[!, name] = Float64.(dbb[!, name])
end
A = dba[:, cols]
B = dbb[:, cols]
C = vcat(A, B)
dist = GowerDF2([:X1,:X4], [:X2,:X3,:X5],C)  
# version pour DataFrameRow
D = pairwise_gower(dist, A, B)

444×2848 Matrix{Float64}:
 0.228518  0.247848   0.00952422  …  0.0235539  0.263835   0.256872
 0.255314  0.0798934  0.24157        0.2556     0.0906304  0.0889176
 0.252918  0.254447   0.0279977      0.0356185  0.288235   0.263471
 0.243368  0.222236   0.0208261      0.0260685  0.278685   0.229819
 0.230995  0.270506   0.0321823      0.046212   0.266312   0.27953
 0.342374  0.282903   0.121226    …  0.107197   0.292525   0.273879
 0.290708  0.269576   0.0657871      0.0734078  0.326025   0.277306
 0.205523  0.0649944  0.226671       0.240701   0.0553726  0.0740185
 0.270901  0.222441   0.0497524      0.0357227  0.234008   0.214858
 0.302531  0.286474   0.0902628      0.082642   0.252681   0.278892
 0.313424  0.0544637  0.292276    …  0.278246   0.0635743  0.046881
 0.230542  0.0504982  0.254287       0.246666   0.019308   0.0429155
 0.236714  0.296186   0.0578625      0.0718921  0.286564   0.30521
 ⋮                                ⋱  ⋮                     
 0.241247  0.237578   0.04136

Format coordonnées continues après FADM

In [12]:

# --- Exemple : DataFrames A et B déjà définis ---
df_all = vcat(A, B)

# --- Colonnes continues et catégorielles ---
continuous_cols = [:X1, :X4]
categorical_cols = [:X2, :X3, :X5]

# 1. Standardisation des variables continues
df_cont_std = DataFrame()
for c in continuous_cols
    x = skipmissing(df_all[!, c])
    μ = mean(x)
    σ = std(x)
    df_cont_std[!, c] = (df_all[!, c] .- μ) ./ σ
end

# 2. One-hot encoding des variables catégorielles
df_dummy = DataFrame()
for c in categorical_cols
    df_cat = categorical(df_all[!, c])
    for lvl in levels(df_cat)
        col_name = Symbol(string(c) * "_" * string(lvl))
        df_dummy[!, col_name] = df_cat .== lvl
    end
end

# 3. Pondération FAMD
df_dummy_famd = DataFrame()
for c in names(df_dummy)
    p = mean(df_dummy[!, c])
    df_dummy_famd[!, c] = df_dummy[!, c] ./ sqrt(p)
end

# 4. Construction de la matrice finale
Xn = hcat(Matrix(df_cont_std), Matrix(df_dummy_famd))

# 5. ACP (PCA)
ncomp = 13
model = fit(PCA, Xn'; maxoutdim=ncomp)

# 6. Coordonnées factorielles
coords_all = MultivariateStats.transform(model, Xn')  #
# --- coords_all contient les coordonnées FAMD de df_all ---
coords_all2 = coords_all'

3292×10 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.0449645  -1.37694   -0.173918   …   0.378294    0.160528   -0.960776
  1.19435    -1.31151   -0.531212       1.25596    -0.695427   -0.473825
  0.389783   -1.32594   -0.197085       0.332067   -0.0128266  -1.46969
  0.420415   -0.78839    0.381008      -0.251398    0.103928   -1.52667
 -0.129282   -1.72846   -0.540338       0.758482    0.111635   -0.84856
 -0.895935    0.204915   1.69723    …  -1.35854     0.906029   -0.0311527
  1.04196    -1.30562   -0.292825       0.323758   -0.286449   -2.2309
  0.20633    -1.42881   -0.480085       1.36254    -0.301792    0.682808
 -0.277194   -0.608515   0.702314      -0.46159     0.445308   -0.720402
 -1.33447    -0.510157   1.00009       -0.590804    0.920607    0.508448
  0.256871    0.285632   1.37216    …  -0.499318    0.0907146   0.556065
 -0.150446   -1.03324    0.0138259      0.925182   -0.0538796   1.08315
 -1.21649    -2.45801   -1.13602        1.52824     0.39975     0.447933


In [13]:
@show size Xn'
@show X

size = size
Xn' = [1.1867004255204339 2.0740929294975 1.6542047690790866 0.9529482537411194 1.565774714675795 -2.040687364205177 2.432760283021323 1.013550668646859 -0.15706536621624093 -1.5997738761663995 -1.281302001916801 0.028452230434008748 1.224422336839328 -1.0747590776461875 -1.783436296850168 -1.41734823946033 0.30672862540938167 1.8823914129588912 -0.2702311001729023 -1.25656632236343 -2.0845931954124093 -1.9367975100810324 0.20716751520709348 1.7253198477950191 0.6474626112570216 2.1105780568387185 -0.9077932406609498 -0.34320135485533904 -1.246053658553259 -1.030234854450129 1.2219487688839987 -0.2863092918825833 2.098210217062021 0.7593915612360063 0.6183981877818175 0.4588530546625933 -0.5528362390701324 -0.7853516268717707 0.6462258272793682 1.1965946973417774 -0.5144959357624104 -1.1502029002839544 0.18057665968721878 0.6938420104195837 1.6616254729450992 -0.06245139192461255 2.232401278639047 2.1600494159454366 0.4205127513548706 1.6634806489116036 2.8322415078081997 -

3292×5 Matrix{Int64}:
 4  4  0  0  5
 4  4  0  0  4
 4  3  0  0  5
 4  1  0  0  5
 4  4  0  0  5
 1  1  0  0  5
 4  1  0  0  5
 4  4  0  0  4
 2  2  0  0  5
 1  4  0  0  5
 1  1  0  0  4
 3  4  0  0  4
 4  4  0  0  5
 ⋮           
 2  4  0  0  6
 1  3  1  1  6
 1  3  1  0  5
 1  3  1  0  6
 4  4  1  0  4
 1  3  1  0  6
 4  4  0  0  6
 3  2  1  0  3
 2  1  0  0  5
 4  3  0  0  5
 2  4  0  0  4
 3  1  0  0  4

In [14]:
nA = nrow(A)
a = coords_all2[1:nA, :]'
b = coords_all2[nA+1:end, :]'
distance = Euclidean()
D = pairwise(distance, a, b, dims = 2)

444×2848 Matrix{Float64}:
 4.27914  2.67426  0.264006  3.30934  2.91712  …  0.638192  2.82378   2.78116
 4.36513  2.18548  2.51415   2.93841  4.07604     2.70121   2.49972   2.41188
 4.50398  2.67809  0.751884  3.55859  3.42535     1.00491   3.27102   2.81867
 4.81433  2.3266   1.15856   3.56879  3.60906     1.01168   3.33133   2.40123
 4.19445  3.11459  0.962004  3.4752   2.9376      1.31737   2.95632   3.24412
 5.68851  3.28797  3.26807   4.08583  3.84689  …  2.895     3.53992   3.10778
 4.99288  3.05914  1.78913   4.1481   4.29264     1.97154   4.09112   3.23435
 4.0897   2.62694  2.71276   2.62878  3.38409     2.84494   1.48983   2.7408
 4.7161   2.32397  1.33944   3.29385  3.04277     0.962295  2.77637   2.28847
 4.94541  3.23872  2.6018    3.54091  2.78107     2.30656   2.65264   3.11194
 5.21217  1.6351   3.34579   2.89131  4.07903  …  3.07816   2.14462   1.38578
 4.18436  2.36444  2.7765    2.36965  3.13498     2.77928   0.547923  2.38564
 4.41238  4.64242  2.96094   4.2152   3

In [17]:

lambda = 0.1
alpha = 0.4
percent_closest = 1
#a = Xmdn[indA, :]'
#b = Xmdn[indB, :]'
#a = X_hot[indA, :]'
#b = X_hot[indB, :]'
distance = Euclidean()
#D = pairwise(distance, a, b, dims = 2)


    # Compute the indexes of individuals with same covariates
indXA = Vector{Int64}[]
indXB = Vector{Int64}[]

Xlevels = unique(eachrow(X))
# aggregate both bases
a = X[indA, :]'
b = X[indB, :]'
for x in Xlevels
        distA = vec(pairwise(distance, x[:, :], a, dims = 2))
        push!(indXA, findall(distA .< 0.1))
end
Xlevels = unique(eachrow(X))
for x in Xlevels
        distB = vec(pairwise(distance, x[:, :], b, dims = 2))
        push!(indXB, findall(distB .< 0.1))
end


norme = Euclidean()

aggregate_tol = 0.5

# Create a model for the optimal transport of individuals
modelA = Model(Clp.Optimizer)
modelB = Model(Clp.Optimizer)
set_optimizer_attribute(modelA, "LogLevel", 0)
set_optimizer_attribute(modelB, "LogLevel", 0)

# Compute data for aggregation of the individuals
Xlevels = eachindex(indXA)

# compute the neighbors of the covariates for regularization
Xvalues = unique(eachrow(Xobserv))
dist_X = pairwise(norme, Xvalues, Xvalues)
voisins = findall.(eachrow(dist_X .<= 1))
nvoisins = length(Xvalues)
C = zeros(Float64, (length(Ylevels), length(Zlevels)))

for y in Ylevels, i in indY[y], z in Zlevels

        nbclose = round(Int, percent_closest * length(indZ[z]))
        if nbclose > 0
            distance = [D[i, j] for j in indZ[z]]
            p = partialsortperm(distance, 1:nbclose)
            C[y, z] += sum(distance[p]) / nbclose / length(indY[y]) / 2.0
        end

end

for z in Zlevels, j in indZ[z], y in Ylevels

        nbclose = round(Int, percent_closest * length(indY[y]))
        if nbclose > 0
            distance = [D[i, j] for i in indY[y]]
            p = partialsortperm(distance, 1:nbclose)
            C[y, z] += sum(distance[p]) / nbclose / length(indZ[z]) / 2.0
        end

end

# Compute the estimators that appear in the model

estim_XA = length.(indXA) ./ nA
estim_XB = length.(indXB) ./ nB
estim_XA_YA = [
        length(indXA[x][findall(Yobserv[indXA[x]] .== y)]) / nA for
            x in Xlevels, y in Ylevels
    ]
estim_XB_ZB = [
        length(indXB[x][findall(Zobserv[indXB[x] .+ nA] .== z)]) / nB for
            x in Xlevels, z in Zlevels
    ]

    # Variables
    # - gammaA[x,y,z]: joint probability of X=x, Y=y and Z=z in base A
@variable(
        modelA,
        gammaA[x in Xlevels, y in Ylevels, z in Zlevels] >= 0,
        base_name = "gammaA"
    )

    # - gammaB[x,y,z]: joint probability of X=x, Y=y and Z=z in base B
@variable(
        modelB,
        gammaB[x in Xlevels, y in Ylevels, z in Zlevels] >= 0,
        base_name = "gammaB"
    )

@variable(modelA, errorA_XY[x in Xlevels, y in Ylevels], base_name = "errorA_XY")
@variable(
        modelA,
        abserrorA_XY[x in Xlevels, y in Ylevels] >= 0,
        base_name = "abserrorA_XY"
    )
@variable(modelA, errorA_XZ[x in Xlevels, z in Zlevels], base_name = "errorA_XZ")
@variable(
        modelA,
        abserrorA_XZ[x in Xlevels, z in Zlevels] >= 0,
        base_name = "abserrorA_XZ"
    )

@variable(modelB, errorB_XY[x in Xlevels, y in Ylevels], base_name = "errorB_XY")
@variable(
        modelB,
        abserrorB_XY[x in Xlevels, y in Ylevels] >= 0,
        base_name = "abserrorB_XY"
    )
@variable(modelB, errorB_XZ[x in Xlevels, z in Zlevels], base_name = "errorB_XZ")
@variable(
        modelB,
        abserrorB_XZ[x in Xlevels, z in Zlevels] >= 0,
        base_name = "abserrorB_XZ"
    )

# Constraints
# - assign sufficient probability to each class of covariates with the same outcome
@constraint(
        modelA,
        ctYandXinA[x in Xlevels, y in Ylevels],
        sum(gammaA[x, y, z] for z in Zlevels) == estim_XA_YA[x, y] + errorA_XY[x, y]
    )
@constraint(
        modelB,
        ctZandXinB[x in Xlevels, z in Zlevels],
        sum(gammaB[x, y, z] for y in Ylevels) == estim_XB_ZB[x, z] + errorB_XZ[x, z]
    )

    # - we impose that the probability of Y conditional to X is the same in the two databases
    # - the consequence is that the probability of Y and Z conditional to Y is also the same in the two bases
@constraint(
        modelA,
        ctZandXinA[x in Xlevels, z in Zlevels],
        estim_XB[x] * sum(gammaA[x, y, z] for y in Ylevels) ==
            estim_XB_ZB[x, z] * estim_XA[x] + estim_XB[x] * errorA_XZ[x, z]
    )

@constraint(
        modelB,
        ctYandXinB[x in Xlevels, y in Ylevels],
        estim_XA[x] * sum(gammaB[x, y, z] for z in Zlevels) ==
            estim_XA_YA[x, y] * estim_XB[x] + estim_XA[x] * errorB_XY[x, y]
    )

    # - recover the norm 1 of the error
@constraint(modelA, [x in Xlevels, y in Ylevels], errorA_XY[x, y] <= abserrorA_XY[x, y])
@constraint(
        modelA,
        [x in Xlevels, y in Ylevels],
        -errorA_XY[x, y] <= abserrorA_XY[x, y]
    )
@constraint(
        modelA,
        sum(abserrorA_XY[x, y] for x in Xlevels, y in Ylevels) <= alpha / 2.0
    )
@constraint(modelA, sum(errorA_XY[x, y] for x in Xlevels, y in Ylevels) == 0.0)
@constraint(modelA, [x in Xlevels, z in Zlevels], errorA_XZ[x, z] <= abserrorA_XZ[x, z])
@constraint(
        modelA,
        [x in Xlevels, z in Zlevels],
        -errorA_XZ[x, z] <= abserrorA_XZ[x, z]
    )

@constraint(
        modelA,
        sum(abserrorA_XZ[x, z] for x in Xlevels, z in Zlevels) <= alpha / 2.0
    )

@constraint(modelA, sum(errorA_XZ[x, z] for x in Xlevels, z in Zlevels) == 0.0)

@constraint(modelB, [x in Xlevels, y in Ylevels], errorB_XY[x, y] <= abserrorB_XY[x, y])

@constraint(
        modelB,
        [x in Xlevels, y in Ylevels],
        -errorB_XY[x, y] <= abserrorB_XY[x, y]
    )

@constraint(
        modelB,
        sum(abserrorB_XY[x, y] for x in Xlevels, y in Ylevels) <= alpha / 2.0
    )

@constraint(modelB, sum(errorB_XY[x, y] for x in Xlevels, y in Ylevels) == 0.0)

@constraint(modelB, [x in Xlevels, z in Zlevels], errorB_XZ[x, z] <= abserrorB_XZ[x, z])

@constraint(
        modelB,
        [x in Xlevels, z in Zlevels],
        -errorB_XZ[x, z] <= abserrorB_XZ[x, z]
    )

@constraint(
        modelB,
        sum(abserrorB_XZ[x, z] for x in Xlevels, z in Zlevels) <= alpha / 2.0
    )

@constraint(modelB, sum(errorB_XZ[x, z] for x in Xlevels, z in Zlevels) == 0.0)

    # - regularization
@variable(
        modelA,
        reg_absA[x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels] >= 0
    )

@constraint(
        modelA,
        [x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels],
        reg_absA[x1, x2, y, z] >=
            gammaA[x1, y, z] / (max(1, length(indXA[x1])) / nA) -
            gammaA[x2, y, z] / (max(1, length(indXA[x2])) / nA)
    )

    @constraint(
        modelA,
        [x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels],
        reg_absA[x1, x2, y, z] >=
            gammaA[x2, y, z] / (max(1, length(indXA[x2])) / nA) -
            gammaA[x1, y, z] / (max(1, length(indXA[x1])) / nA)
    )

@expression(
        modelA,
        regterm,
        sum(
            1 / nvoisins * reg_absA[x1, x2, y, z] for
                x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels
        )
    )

@variable(
        modelB,
        reg_absB[x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels] >= 0
    )

@constraint(
        modelB,
        [x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels],
        reg_absB[x1, x2, y, z] >=
            gammaB[x1, y, z] / (max(1, length(indXB[x1])) / nB) -
            gammaB[x2, y, z] / (max(1, length(indXB[x2])) / nB)
    )

@constraint(
        modelB,
        [x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels],
        reg_absB[x1, x2, y, z] >=
            gammaB[x2, y, z] / (max(1, length(indXB[x2])) / nB) -
            gammaB[x1, y, z] / (max(1, length(indXB[x1])) / nB)
    )

@expression(
        modelB,
        regterm,
        sum(
            1 / nvoisins * reg_absB[x1, x2, y, z] for
                x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels
        )
    )

    # by default, the OT cost and regularization term are weighted to lie in the same interval
@objective(
        modelA,
        Min,
        sum(C[y, z] * gammaA[x, y, z] for y in Ylevels, z in Zlevels, x in Xlevels) +
            lambda * sum(
            1 / nvoisins * reg_absA[x1, x2, y, z] for
                x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels
        )
    )

@objective(
        modelB,
        Min,
        sum(C[y, z] * gammaB[x, y, z] for y in Ylevels, z in Zlevels, x in Xlevels) +
            lambda * sum(
            1 / nvoisins * reg_absB[x1, x2, y, z] for
                x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels
        )
    )

    # Solve the problem
optimize!(modelA)
optimize!(modelB)

    # Extract the values of the solution
gammaA_val = [value(gammaA[x, y, z]) for x in Xlevels, y in Ylevels, z in Zlevels]
gammaB_val = [value(gammaB[x, y, z]) for x in Xlevels, y in Ylevels, z in Zlevels]

    # compute the resulting estimators for the distributions of Z
    # conditional to X and Y in base A and of Y conditional to X and Z in base B

estimatorZA = ones(length(Xlevels), length(Ylevels), length(Zlevels)) ./ length(Zlevels)

for x in Xlevels, y in Ylevels
        proba_c_mA = sum(gammaA_val[x, y, Zlevels])
        if proba_c_mA > 1.0e-6
            estimatorZA[x, y, :] = gammaA_val[x, y, :] ./ proba_c_mA
        end
end

estimatorYB = ones(length(Xlevels), length(Ylevels), length(Zlevels)) ./ length(Ylevels)

for x in Xlevels, z in Zlevels
        proba_c_mB = sum(view(gammaB_val, x, Ylevels, z))
        if proba_c_mB > 1.0e-6
            estimatorYB[x, :, z] = view(gammaB_val, x, :, z) ./ proba_c_mB
        end
end

nbX = length(indXA)

    # Count the number of mistakes in the transport
    #deduce the individual distributions of probability for each individual from the distributions
probaZindivA = zeros(Float64, (nA, length(Zlevels)))
probaYindivB = zeros(Float64, (nB, length(Ylevels)))
for x in 1:nbX
        for i in indXA[x]
            probaZindivA[i, :] .= estimatorZA[x, Yobserv[i], :]
end
for i in indXB[x]
            probaYindivB[i, :] .= estimatorYB[x, :, Zobserv[i + nA]]
        end
end

    # Transport the modality that maximizes frequency
predZA = [findmax([probaZindivA[i, z] for z in Zlevels])[2] for i in 1:nA]
predYB = [findmax([probaYindivB[j, y] for y in Ylevels])[2] for j in 1:nB]



2848-element Vector{Int64}:
 1
 3
 1
 4
 1
 4
 2
 1
 1
 1
 1
 1
 4
 ⋮
 1
 1
 1
 1
 1
 1
 1
 4
 3
 3
 4
 3

In [18]:

tab1 = FreqTables.freqtable(dba.Y,predZA)
tab2 = FreqTables.freqtable(dbb.Z,predYB)
@show round.(tab1 ./ sum(tab1) .* 100, digits = 2)
#@show round.(tab2 ./ sum(tab2) .* 100, digits = 2)
#deuxieme essai avec CSP
#Y codage timoun 1 2 3 4 5
#Y1 primaire ==> Z1
#Y2 secondaire ==> Z2
#Y3 baccalaureat ==> Z3
#Y4 supérieures ==> Z4 Z5 Z6

#accZ=(tab1[1,1]+tab1[2,2]+tab1[3,3]+tab1[4,4]+tab1[4,5])/sum(tab1)
#accZ=(tab1[1,1]+tab1[2,2]+tab1[3,3]+tab1[4,4]+tab1[4,6])/sum(tab1)
#accY=(tab2[1,1]+tab2[2,2]+tab2[3,3]+tab2[4,4]+tab2[5,4]+tab2[6,4])/sum(tab2)
#@show accZ

round.((tab1 ./ sum(tab1)) .* 100, digits = 2) = [0.0 0.0 1.13 0.0 0.0 0.0; 3.83 21.85 9.68 0.0 1.13 0.68; 0.45 0.23 3.83 0.0 17.12 0.0; 0.0 0.0 0.23 9.01 6.08 24.77]


4×6 Named Matrix{Float64}
Dim1 ╲ Dim2 │     1      2      3      4      5      6
────────────┼─────────────────────────────────────────
1           │   0.0    0.0   1.13    0.0    0.0    0.0
2           │  3.83  21.85   9.68    0.0   1.13   0.68
3           │  0.45   0.23   3.83    0.0  17.12    0.0
4           │   0.0    0.0   0.23   9.01   6.08  24.77

In [19]:
predZAm = [findmax([probaZindivA[i, z] for z in Zlevels])[1] for i in 1:nA]
predYBm = [findmax([probaYindivB[j, y] for y in Ylevels])[1] for j in 1:nB]

df1 = DataFrame(Y=dba.Y, pred=predZA, prob=predZAm)

# Groupement par combinaison (Y, pred)
stats1 = combine(groupby(df1, [:Y, :pred]),
                 :prob => (x -> round(mean(x), digits=2)) => :mean_prob,
                 :prob => (x -> round(std(x),  digits=2)) => :std_prob)

Row,Y,pred,mean_prob,std_prob
,Int64,Int64,Float64,Float64
1,1,3,0.67,0.18
2,2,1,0.17,0.0
3,2,2,0.6,0.11
4,2,3,0.63,0.14
5,2,5,0.64,0.1
6,2,6,1.0,0.0
7,3,1,0.17,0.0
8,3,2,1.0,NaN
9,3,3,0.59,0.11


In [20]:
df1 = DataFrame(Y=dbb.Z, pred=predYB, prob=predYBm)

# Groupement par combinaison (Y, pred)
stats1 = combine(groupby(df1, [:Y, :pred]),
                 :prob => (x -> round(mean(x), digits=2)) => :mean_prob,
                 :prob => (x -> round(std(x),  digits=2)) => :std_prob)

Row,Y,pred,mean_prob,std_prob
,Int64,Int64,Float64,Float64
1,1,1,0.55,0.41
2,1,2,1.0,0.0
3,1,4,1.0,NaN
4,2,1,0.83,0.26
5,2,2,0.63,0.1
6,2,3,0.58,0.2
7,2,4,1.0,0.0
8,3,1,0.72,0.18
9,3,2,0.53,0.13


In [21]:
df1 = DataFrame(Y=dba.Y, pred=predZA, prob=predZAm)

# Groupement par combinaison (Y, pred)
stats1 = combine(groupby(df1, [:Y, :pred]),
                 :prob => (x -> round(mean(x), digits=2)) => :mean_prob,
                 :prob => (x -> round(std(x),  digits=2)) => :std_prob)

Row,Y,pred,mean_prob,std_prob
,Int64,Int64,Float64,Float64
1,1,3,0.67,0.18
2,2,1,0.17,0.0
3,2,2,0.6,0.11
4,2,3,0.63,0.14
5,2,5,0.64,0.1
6,2,6,1.0,0.0
7,3,1,0.17,0.0
8,3,2,1.0,NaN
9,3,3,0.59,0.11


In [119]:
@show(data)

Row,X1,X2,X3,X4,X5,Y,Z,database
,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,36.3258,0.0,0.0,24.16,5.0,3,-1,1
2,40.2546,0.0,0.0,24.8,4.0,4,-1,1
3,38.3956,0.0,0.0,21.99,5.0,2,-1,1
4,35.2909,0.0,0.0,18.65,5.0,3,-1,1
5,38.0041,0.0,0.0,26.62,5.0,2,-1,1
6,22.037,0.0,0.0,18.37,5.0,4,-1,1
7,41.8426,0.0,0.0,19.07,5.0,2,-1,1
8,35.5592,0.0,0.0,29.74,4.0,2,-1,1
9,30.3765,0.0,0.0,20.57,5.0,2,-1,1


In [ ]:
OT -between

In [476]:
T = Float64
X_hot=Matrix{T}(X_hot)
Xdf = DataFrame(X_hot, Symbol.("X" .* string.(1:size(X_hot, 2))))
data2 = hcat(Xdf,data[:, [:Y, :Z, :database]])
dba = subset(data2, :database => ByRow(==(1)))
dbb = subset(data2, :database => ByRow(==(2)))
@show data2

T = Float64
Xdf = DataFrame(coords_all2, Symbol.("X" .* string.(1:size(coords_all2, 2))))
data3 = hcat(Xdf,data[:, [:Y, :Z, :database]])
dba = subset(data3, :database => ByRow(==(1)))
dbb = subset(data3, :database => ByRow(==(2)))
@show data3


Row,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y,Z,database
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,-0.0449645,-1.37694,-0.173918,-0.392016,0.00129394,0.604896,0.461918,0.378294,0.160528,-0.960776,3,-1,1
2,1.19435,-1.31151,-0.531212,0.85059,-0.399683,-0.164033,0.0852383,1.25596,-0.695427,-0.473825,4,-1,1
3,0.389783,-1.32594,-0.197085,-0.473297,0.00129394,0.604896,0.397655,0.332067,-0.0128266,-1.46969,2,-1,1
4,0.420415,-0.78839,0.381008,-0.571249,0.00129394,0.604896,0.00594869,-0.251398,0.103928,-1.52667,3,-1,1
5,-0.129282,-1.72846,-0.540338,-0.317246,0.00129394,0.604896,0.722101,0.758482,0.111635,-0.84856,2,-1,1
6,-0.895935,0.204915,1.69723,-0.523372,0.00129394,0.604896,-0.631684,-1.35854,0.906029,-0.0311527,4,-1,1
7,1.04196,-1.30562,-0.292825,-0.585527,0.00129394,0.604896,0.341983,0.323758,-0.286449,-2.2309,2,-1,1
8,0.20633,-1.42881,-0.480085,1.03556,-0.399683,-0.164033,0.232301,1.36254,-0.301792,0.682808,2,-1,1
9,-0.277194,-0.608515,0.702314,-0.486023,0.00129394,0.604896,-0.0806953,-0.46159,0.445308,-0.720402,2,-1,1


In [448]:
@show data3

Row,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y,Z,database
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,-0.0449645,-1.37694,-0.173918,-0.392016,0.00129394,0.604896,0.461918,0.378294,0.160528,-0.960776,3,-1,1
2,1.19435,-1.31151,-0.531212,0.85059,-0.399683,-0.164033,0.0852383,1.25596,-0.695427,-0.473825,4,-1,1
3,0.389783,-1.32594,-0.197085,-0.473297,0.00129394,0.604896,0.397655,0.332067,-0.0128266,-1.46969,2,-1,1
4,0.420415,-0.78839,0.381008,-0.571249,0.00129394,0.604896,0.00594869,-0.251398,0.103928,-1.52667,3,-1,1
5,-0.129282,-1.72846,-0.540338,-0.317246,0.00129394,0.604896,0.722101,0.758482,0.111635,-0.84856,2,-1,1
6,-0.895935,0.204915,1.69723,-0.523372,0.00129394,0.604896,-0.631684,-1.35854,0.906029,-0.0311527,4,-1,1
7,1.04196,-1.30562,-0.292825,-0.585527,0.00129394,0.604896,0.341983,0.323758,-0.286449,-2.2309,2,-1,1
8,0.20633,-1.42881,-0.480085,1.03556,-0.399683,-0.164033,0.232301,1.36254,-0.301792,0.682808,2,-1,1
9,-0.277194,-0.608515,0.702314,-0.486023,0.00129394,0.604896,-0.0806953,-0.46159,0.445308,-0.720402,2,-1,1


In [479]:
@show X

3292×5 Matrix{Int64}:
 4  4  0  0  5
 4  4  0  0  4
 4  3  0  0  5
 4  1  0  0  5
 4  4  0  0  5
 1  1  0  0  5
 4  1  0  0  5
 4  4  0  0  4
 2  2  0  0  5
 1  4  0  0  5
 ⋮           
 1  3  1  0  6
 4  4  1  0  4
 1  3  1  0  6
 4  4  0  0  6
 3  2  1  0  3
 2  1  0  0  5
 4  3  0  0  5
 2  4  0  0  4
 3  1  0  0  4

In [ ]:
X=Matrix{T}(X)
Xdf = DataFrame(X, Symbol.("X" .* string.(1:size(X, 2))))
data4 = hcat(Xdf,data[:, [:Y, :Z, :database]])


Row,X1,X2,X3,X4,X5,Y,Z,database
,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,4.0,4.0,0.0,0.0,5.0,3,-1,1
2,4.0,4.0,0.0,0.0,4.0,4,-1,1
3,4.0,3.0,0.0,0.0,5.0,2,-1,1
4,4.0,1.0,0.0,0.0,5.0,3,-1,1
5,4.0,4.0,0.0,0.0,5.0,2,-1,1
6,1.0,1.0,0.0,0.0,5.0,4,-1,1
7,4.0,1.0,0.0,0.0,5.0,2,-1,1
8,4.0,4.0,0.0,0.0,4.0,2,-1,1
9,2.0,2.0,0.0,0.0,5.0,2,-1,1


In [506]:
A= otrecod(data2, JointOTBetweenBases(reg = 0.001,
reg_m1= 0.01,
reg_m2 = 0.01,
Ylevels = 1:4,
Zlevels = 1:6))

@show A.yb_pred
@show A.za_pred

444-element Vector{Int64}:
 2
 6
 3
 2
 2
 3
 2
 6
 5
 3
 ⋮
 6
 3
 6
 6
 3
 2
 6
 6
 5

In [498]:
A= otrecod(data3, JointOTBetweenBasesWithPredictors(reg = 0.001,
reg_m1= 0.01,
reg_m2 = 0.01,
Ylevels = 1:4,
Zlevels = 1:6))



OptimalTransportDataIntegration.JointOTResult([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1  …  -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1  …  -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [2, 4, 2, 4, 2, 2, 2, 4, 4, 2  …  4, 2, 4, 2, 2, 4, 2, 2, 4, 4], [5, 6, 6, 5, 5, 5, 6, 6, 5, 5  …  5, 6, 5, 6, 6, 5, 5, 6, 6, 5])

In [510]:
tab1 = FreqTables.freqtable(dba.Y,A.za_pred)
tab2 = FreqTables.freqtable(dbb.Z,A.yb_pred)
@show tab2
#accZ=(tab1[1,1]+tab1[2,2]+tab1[3,3]+tab1[4,4]+tab1[4,5])/sum(tab1)
#accY=(tab2[1,1]+tab2[2,2]+tab2[3,3]+tab2[4,4]+tab2[5,4]+tab2[6,4])/sum(tab2)
@show tab2
accZ=(tab1[1,1]+tab1[2,2]+tab1[3,3]+tab1[4,4]+tab1[4,5])/sum(tab1)


0.5698198198198198

OT-between à la main

In [ ]:
dba = subset(data, :database => ByRow(==(1)))
dbb = subset(data, :database => ByRow(==(2)))

colnames = names(data, r"^X")
XA = transpose(Matrix{Float32}(dba[!, colnames]))
XB = transpose(Matrix{Float32}(dbb[!, colnames]))

YA = Flux.onehotbatch(dba.Y, Ylevels)
ZB = Flux.onehotbatch(dbb.Z, Zlevels)

XYA = vcat(XA, YA)
XZB = vcat(XB, ZB)
dimXYA = size(XYA, 1)
dimXZB = size(XZB, 1)
nA = size(dba, 1)
nB = size(dbb, 1)

wa = ones(Float32, nA) ./ nA
wb = ones(Float32, nB) ./ nB
iterations = 10
learning_rate = 0.01
batchsize = 444
epochs = 1000
hidden_layer_size = 10


10

Transport des covariables - choix du rho optimal

In [ ]:
dba = subset(data, :database => ByRow(==(1)))
dbb = subset(data, :database => ByRow(==(2)))
XA1=dba[:,1]
XA2=dba[:,2].+1
XA3=dba[:,3].+1
XA4=dba[:,4]
XA5=dba[:,5]
XA2f = Flux.onehotbatch(XA2, 1:2)
XA3f = Flux.onehotbatch(XA3, 1:2)
XA5f = Flux.onehotbatch(XA5, 1:8)
YA = Flux.onehotbatch(dba.Y, Ylevels)
ZB = Flux.onehotbatch(dbb.Z, Zlevels)

reg_m1_list = [0.01, 0.1,0.5, 1.0]
reg_m2_list = [0.01, 0.1, 0.5,1.0]

results = []   # vecteur vide pour stocker les tuples (reg_m1, reg_m2, cost)
cols = names(dba, r"^X") 
A = dba[:, cols]
B = dbb[:, cols]
C = vcat(A, B)
dist = GowerDF2([:X1,:X4], [:X2,:X3,:X5],C)  
 
C0 = Float32.(pairwise_gower(dist, dba, dbb))
C = C0.^2 ./ maximum(C0.^2)

nA = size(dba, 1)
nB = size(dbb, 1)

wa = ones(Float32, nA) ./ nA
wb = ones(Float32, nB) ./ nB
print(size(XA1))


In [485]:
@show C

444×2848 Matrix{Float32}:
 0.0685128  0.0805936   0.000119012  0.253548   …  0.0913262    0.0865693
 0.0855223  0.0083744   0.0765629    0.0931126     0.0107766    0.0103731
 0.0839251  0.0849425   0.00102843   0.282476      0.109        0.0910744
 0.0777069  0.064798    0.000569044  0.270968      0.101897     0.0692952
 0.0700066  0.0960029   0.00135884   0.273152      0.0930496    0.102515
 0.153792   0.105004    0.0192808    0.324235   …  0.112268     0.0984121
 0.110878   0.0953438   0.00567823   0.33036       0.139454     0.100891
 0.0554182  0.00554221  0.06741      0.0825083     0.00402274   0.00718808
 0.0962834  0.0649173   0.00324758   0.237703      0.0718446    0.0605668
 0.12008    0.107672    0.0106893    0.274344      0.0837678    0.102048
 ⋮                                              ⋱               
 0.0769063  0.00448809  0.0635955    0.0841124  …  0.00786557   0.00597988
 0.115041   0.0978426   0.00775331   0.266699      0.0795677    0.0924845
 0.0861318  0.0762114 

In [44]:
XB1=dbb[:,1]
XB2=dbb[:,2].+1
XB3=dbb[:,3].+1
XB4=dbb[:,4]
XB5=dbb[:,5]
XB2f = Flux.onehotbatch(XB2, 1:2)
XB3f = Flux.onehotbatch(XB3, 1:2)
XB5f = Flux.onehotbatch(XB5, 1:8)

8×2848 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  1  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1  1
 ⋅  ⋅  1  ⋅  1  1  1  ⋅  ⋅  ⋅  1  1  ⋅     ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1  1  ⋅  ⋅
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  …  1  1  ⋅  1  ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

Choix du paramètre rho optimal

In [179]:
# listes de valeurs
reg_m1_list = [0.01, 0.1,0.5, 1.0]
reg_m2_list = [0.01, 0.1, 0.5,1.0]
#reg_m1_list = [0.01]
#reg_m2_list = [0.01]
reg= 0.001
results = []   # vecteur vide pour stocker les tuples (reg_m1, reg_m2, cost)
XBpred1 = Vector{Float64}[]
XBpred2 = Matrix{Float64}[]
XBpred3 = Matrix{Float64}[]
XBpred4 = Vector{Float64}[]
XBpred5 = Matrix{Float64}[]
XBpred = Matrix{Float64}[]
XB = DataFrame(
    X1 = XB1,
    X2 = XB2,
    X3 = XB3,
    X4 = XB4,
    X5 = XB5
)
for r1 in reg_m1_list
    for r2 in reg_m2_list
        G = PythonOT.mm_unbalanced(wa, wb, C, (r1, r2); reg = reg, div = "kl")
        row_sumsB = vec(sum(G, dims=2))
        col_sumsA = vec(sum(G, dims=1))
        XBpred1 = (1 ./col_sumsA) .* vec(XA1' * G)
        XBpred2f = (XA2f* G) .* (1 ./col_sumsA)'
        XBpred2 = Flux.onecold(XBpred2f)
        XBpred3f = (XA3f* G) .* (1 ./col_sumsA)'
        XBpred3 = Flux.onecold(XBpred3f)
        XBpred4 =(1 ./col_sumsA) .* vec(XA4' * G)
        XBpred5f = (XA5f* G) .* (1 ./col_sumsA)'
        XBpred5 = Flux.onecold(XBpred5f)
        XBpred = DataFrame(
                X1 = XBpred1,
                X2 = XBpred2,
                X3 = XBpred3,
                X4 = XBpred4,
                X5 = XBpred5
                )
        cost=0
		for i in 1:nB
            cost += Distances.evaluate(dist, XB[i,:], XBpred[i,:])
        end
        push!(results, (reg_m1=r1, reg_m2=r2, cost=cost))
    end
end
best = findmin([r.cost for r in results])  # retourne (valeur_min, index)
best_value, idx = best
best_params = results[idx]

println("Meilleur coût = ", best_value)
println("Paramètres optimaux = ", best_params)


In [180]:
best_params

(reg_m1 = 0.01, reg_m2 = 0.01, cost = 1158.1999999999907)

In [181]:
@show XBpred

Row,X1,X2,X3,X4,X5
,Float64,Int64,Int64,Float64,Int64
1,33.23,1,1,24.3716,5
2,32.984,1,1,24.6907,4
3,33.559,1,1,24.6307,5
4,33.1069,1,1,24.4929,4
5,33.5707,1,1,24.4397,5
6,33.5707,1,1,24.4397,5
7,33.559,1,1,24.6307,5
8,35.3073,1,1,27.92,5
9,33.1069,1,1,24.4929,4


In [511]:
r1= 0.01
r2= 0.01
reg= 0.001
G = PythonOT.mm_unbalanced(wa, wb, C, (r1, r2); reg = reg, div = "kl")
row_sumsB = vec(sum(G, dims=2))
col_sumsA = vec(sum(G, dims=1))

2848-element Vector{Float32}:
 0.00028106605
 0.00030761783
 0.0006368134
 9.2869115f-5
 9.434407f-5
 9.120554f-5
 0.0005519456
 8.678409f-5
 0.00011010439
 0.00028502033
 ⋮
 6.2700004f-5
 0.00010784036
 6.815408f-5
 0.0003089127
 0.000102046586
 0.000472533
 0.00058970915
 0.00031083138
 0.00029370203

In [512]:
XBpred1 = (1 ./col_sumsA) .* vec(XA1' * G)
XBpred2f = (XA2f* G) .* (1 ./col_sumsA)'
XBpred2 = Flux.onecold(XBpred2f)
XBpred3f = (XA3f* G) .* (1 ./col_sumsA)'
XBpred3 = Flux.onecold(XBpred3f)
XBpred4 =(1 ./col_sumsA) .* vec(XA4' * G)
XBpred5f = (XA5f* G) .* (1 ./col_sumsA)'
XBpred5 = Flux.onecold(XBpred5f)

2848-element Vector{Int64}:
 6
 4
 5
 4
 5
 5
 5
 4
 4
 6
 ⋮
 6
 4
 6
 6
 3
 5
 5
 4
 4

In [513]:

XApred1 = (1 ./row_sumsB) .* vec(XB1' * G')
XApred2f = (XB2f* G') .* (1 ./row_sumsB)'
XApred2 = Flux.onecold(XApred2f)
XApred3f = (XB3f* G') .* (1 ./row_sumsB)'
XApred3 = Flux.onecold(XApred3f)
XApred4 =(1 ./row_sumsB) .* vec(XB4' * G')
XApred5f = (XB5f* G') .* (1 ./row_sumsB)'
XApred5 = Flux.onecold(XApred5f)

444-element Vector{Int64}:
 5
 4
 5
 5
 5
 5
 5
 4
 5
 5
 ⋮
 4
 5
 3
 4
 5
 5
 4
 4
 5

In [514]:
XAt = hcat(XA1,XA2f',XA3f',XA4,XA5f')
XBpredt = hcat(XBpred1,XBpred2f',XBpred3f',XBpred4,XBpred5f')
XA=XAt'
XBpred=XBpredt'
XBt = hcat(XB1,XB2f',XB3f',XB4,XB5f')
XApredt = hcat(XApred1,XApred2f',XApred3f',XApred4,XApred5f')
XB=XBt'
XApred=XApredt'

14×444 adjoint(::Matrix{Float64}) with eltype Float64:
 33.2294       37.4873       34.6104       …  30.4934       29.4088
  1.0           1.0           1.0              1.0           1.0
  1.4592e-19    1.02303e-19   1.03084e-19      5.88202e-21   2.38602e-19
  1.0           1.0           1.0              1.0           1.0
  9.40174e-22   3.58763e-19   9.7828e-22       1.28961e-21   1.16891e-19
 22.1653       22.0318       21.2244       …  22.2889       20.1261
  3.61751e-24   1.45827e-24   2.28416e-18      1.33149e-29   1.40297e-25
  8.83498e-29   1.05064e-34   6.3069e-27       7.88437e-34   8.28878e-25
  2.34551e-20   7.32169e-26   4.90347e-20      2.84843e-22   5.15536e-20
  1.08565e-23   1.0           2.34333e-23      1.0           7.29943e-23
  1.0           5.03242e-29   1.0          …   1.89469e-28   1.0
  2.40601e-23   5.1351e-28    8.23467e-24      5.7297e-27    2.18309e-23
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0     

In [7]:
dimXA = size(XA, 1)
dimXB = size(XB, 1)
dimYA = size(YA, 1)
dimZB = size(ZB, 1)

modelXYA = Chain(Dense(dimXA, hidden_layer_size), Dense(hidden_layer_size, dimYA))
modelXZB = Chain(Dense(dimXB, hidden_layer_size), Dense(hidden_layer_size, dimZB))

function train!(model, x, y)

        loader = Flux.DataLoader((x, y), batchsize = batchsize, shuffle = true)
        optim = Flux.setup(Flux.Adam(learning_rate), model)

        for epoch in 1:epochs
            for (x, y) in loader
                grads = Flux.gradient(model) do m
                    y_hat = m(x)
                    Flux.logitcrossentropy(y_hat, y)
                end
                Flux.update!(optim, model, grads[1])
            end
        end

        return
end


train! (generic function with 1 method)

In [9]:
train!(modelXYA, XApred, YA)
train!(modelXZB, XBpred, ZB)
ZApred = modelXZB(XA)
YBpred = modelXYA(XB)

ybpred=Flux.onecold(YBpred)
zapred= Flux.onecold(ZApred)

UndefVarError: UndefVarError: `XApred` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [523]:
using Pkg
using FreqTables
tab1 = FreqTables.freqtable(dba.Y,zapred)
tab2 = FreqTables.freqtable(dbb.Z,ybpred)
@show tab2
#accZ=(tab1[2,1]+tab1[3,2]+tab1[4,3]+tab1[4,4])/sum(tab1)
accY=(tab2[1,1]+tab2[2,2]+tab2[3,3]+tab2[4,4]+tab2[5,4]+tab2[6,4])/sum(tab2)


0.6453651685393258

Avec la distance de gower

In [37]:
function loss_crossentropy(Y, F)

        ϵ = 1.0e-12
        res = zeros(Float32, size(Y, 2), size(F, 2))
        logF = zeros(Float32, size(F))

        for i in eachindex(F)
            if F[i] ≈ 1.0
                logF[i] = log(1.0 - ϵ)
            else
                logF[i] = log(ϵ)
            end
        end

        for i in axes(Y, 1)
            res .+= -Y[i, :] .* logF[i, :]'
        end

        return res

    end

loss_crossentropy (generic function with 1 method)

In [49]:
dimXA = size(XA, 1)
dimXB = size(XB, 1)
dimYA = size(YA, 1)
dimZB = size(ZB, 1)
XA = transpose(Matrix{Float32}(dba[!, colnames]))
XB = transpose(Matrix{Float32}(dbb[!, colnames]))
XYA = vcat(XA, YA)
XZB = vcat(XB, ZB)
dimXYA = size(XYA, 1)
dimXZB = size(XZB, 1)
modelXYA = Chain(Dense(dimXYA, hidden_layer_size), Dense(hidden_layer_size, dimZB))
modelXZB = Chain(Dense(dimXZB, hidden_layer_size), Dense(hidden_layer_size, dimYA))
YBpred = modelXZB(XZB)
ZApred = modelXYA(XYA)
function train!(model, x, y)

        loader = Flux.DataLoader((x, y), batchsize = batchsize, shuffle = true)
        optim = Flux.setup(Flux.Adam(learning_rate), model)

        for epoch in 1:epochs
            for (x, y) in loader
                grads = Flux.gradient(model) do m
                    y_hat = m(x)
                    Flux.logitcrossentropy(y_hat, y)
                end
                Flux.update!(optim, model, grads[1])
            end
        end

        return
end

G = ones(Float32, nA, nB)

results = []   # vecteur vide pour stocker les tuples (reg_m1, reg_m2, cost)
cols = names(dba, r"^X") 
A = dba[:, cols]
B = dbb[:, cols]
C = vcat(A, B)
dist = GowerDF2([:X1,:X4], [:X2,:X3,:X5],C)  

C0 = Float32.(pairwise_gower(dist, A, B))

C = C0.^2 ./ maximum(C0.^2)
cost = Inf
reg = 0.001
reg_m1 = 0.01
reg_m2 = 0.01

0.01

In [56]:
wa = ones(Float32, nA) ./ nA
wb = ones(Float32, nB) ./ nB

alpha1, alpha2 = 1 / length(Ylevels), 1 / length(Zlevels)

G = ones(Float32, nA, nB)
cost = Inf

YB = nB .* YA * G
ZA = nA .* ZB * G'
for iter in 1:iterations # BCD algorithm

        Gold = copy(G)
        costold = cost
        G .= PythonOT.emd(wa, wb, C)
        if reg > 0
            G .= PythonOT.mm_unbalanced(wa, wb, C, (reg_m1, reg_m2); reg = reg, div = "kl")
        else
            G .= PythonOT.emd(wa, wb, C)
        end

        delta = norm(G .- Gold)
        #row_sumsB = vec(sum(G, dims=2))
        #col_sumsA = vec(sum(G, dims=1))
        #YB .=  (YA * G).* (1 ./col_sumsA)'
        #ZA .=  (ZB * G').* (1 ./row_sumsB)'
        YB .= nB .* YA * G
        ZA .= nA .* ZB * G'
        train!(modelXYA, XYA, ZA)
        train!(modelXZB, XZB, YB)

        YBpred .= modelXZB(XZB)
        ZApred .= modelXYA(XYA)

        loss_y = alpha1 * loss_crossentropy(YA, YBpred)
        loss_z = alpha2 * loss_crossentropy(ZB, ZApred)

        fcost = loss_y .^ 2 .+ loss_z' .^ 2

        cost = sum(G .* fcost)


        C .= C0 .+ fcost

end

ybpred=Flux.onecold(YBpred)
zapred= Flux.onecold(ZApred)



444-element Vector{Int64}:
 5
 6
 5
 5
 5
 5
 5
 5
 3
 5
 ⋮
 6
 5
 6
 6
 5
 5
 6
 6
 5

In [61]:
tab1 = FreqTables.freqtable(dba.Y,zapred)
tab2 = FreqTables.freqtable(dbb.Z,ybpred)
@show tab1

#accZ=(tab1[2,1]+tab1[3,2]+tab1[4,3]+tab1[4,4])/sum(tab1)
#accY=(tab2[1,1]+tab2[2,2]+tab2[3,3]+tab2[4,4]+tab2[5,4]+tab2[6,4])/sum(tab2)

#accY=(tab2[1,1]+tab2[2,2]+tab2[4,3]+tab2[5,3]+tab2[6,3])/sum(tab2)


4×4 Named Matrix{Int64}
Dim1 ╲ Dim2 │   2    3    5    6
────────────┼───────────────────
1.0         │   0    1    4    0
2.0         │   5   52   97   11
3.0         │   1   19   59   17
4.0         │   0    0   74  104